# Classification Project

# SetUp

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

pd.set_option("display.max_columns", 300)
pd.set_option("display.max_info_columns", 300)

application_train = pd.read_csv("/Users/drewpeterson/Documents/Programming/Python/LFZ_Python/LFZ_Week12/Classification_Project/home-credit-default-risk/application_train.csv")
application_test = pd.read_csv("/Users/drewpeterson/Documents/Programming/Python/LFZ_Python/LFZ_Week12/Classification_Project/home-credit-default-risk/application_test.csv")
credit_card_balance = pd.read_csv("/Users/drewpeterson/Documents/Programming/Python/LFZ_Python/LFZ_Week12/Classification_Project/home-credit-default-risk/credit_card_balance.csv")
installments_payments = pd.read_csv("/Users/drewpeterson/Documents/Programming/Python/LFZ_Python/LFZ_Week12/Classification_Project/home-credit-default-risk/installments_payments.csv")
previous_application = pd.read_csv("/Users/drewpeterson/Documents/Programming/Python/LFZ_Python/LFZ_Week12/Classification_Project/home-credit-default-risk/previous_application.csv")
POS_CASH_balance = pd.read_csv("/Users/drewpeterson/Documents/Programming/Python/LFZ_Python/LFZ_Week12/Classification_Project/home-credit-default-risk/POS_CASH_balance.csv")
bureau = pd.read_csv("/Users/drewpeterson/Documents/Programming/Python/LFZ_Python/LFZ_Week12/Classification_Project/home-credit-default-risk/bureau.csv")
bureau_balance = pd.read_csv("/Users/drewpeterson/Documents/Programming/Python/LFZ_Python/LFZ_Week12/Classification_Project/home-credit-default-risk/bureau_balance.csv")

# ONE at a Time

## Installments Payments

In [2]:
installments_payments["DAYS_DIFFERENCE"] = installments_payments.DAYS_INSTALMENT - installments_payments.DAYS_ENTRY_PAYMENT
installments_payments["AMT_DIFFERENCE"] = installments_payments.AMT_PAYMENT - installments_payments.AMT_INSTALMENT

installments_payments.drop(columns=["NUM_INSTALMENT_NUMBER", "DAYS_INSTALMENT", "DAYS_ENTRY_PAYMENT", "AMT_INSTALMENT", "AMT_PAYMENT"],inplace= True)
installments_payments.dropna(inplace=True)

installments_payments.NUM_INSTALMENT_VERSION = installments_payments.groupby(["SK_ID_PREV","SK_ID_CURR"]).NUM_INSTALMENT_VERSION.transform(lambda x: x.median())
installments_payments.DAYS_DIFFERENCE = installments_payments.groupby(["SK_ID_PREV","SK_ID_CURR"]).DAYS_DIFFERENCE.transform(lambda x: x.mean())
installments_payments.AMT_DIFFERENCE = installments_payments.groupby(["SK_ID_PREV","SK_ID_CURR"]).AMT_DIFFERENCE.transform(lambda x: x.mean())

installments_payments.drop_duplicates(inplace=True)

In [54]:
installments_payments.isnull().sum()

SK_ID_PREV                0
SK_ID_CURR                0
NUM_INSTALMENT_VERSION    0
DAYS_DIFFERENCE           0
AMT_DIFFERENCE            0
dtype: int64

## Credit Card Balance

In [3]:
credit_card_balance = credit_card_balance.fillna(0)

credit_card_balance["AMT_DRAWINGS_TOTAL"] = credit_card_balance.AMT_DRAWINGS_ATM_CURRENT + credit_card_balance.AMT_DRAWINGS_OTHER_CURRENT + credit_card_balance.AMT_DRAWINGS_CURRENT + credit_card_balance.AMT_DRAWINGS_POS_CURRENT
credit_card_balance["CNT_DRAWINGS_TOTAL"] = credit_card_balance.CNT_DRAWINGS_ATM_CURRENT + credit_card_balance.CNT_DRAWINGS_OTHER_CURRENT + credit_card_balance.CNT_DRAWINGS_CURRENT + credit_card_balance.CNT_DRAWINGS_POS_CURRENT
credit_card_balance["COUNT_CREDIT_CARD"] = credit_card_balance.groupby(["SK_ID_PREV","SK_ID_CURR"]).AMT_BALANCE.transform(lambda x: x.count())

credit_card_balance = credit_card_balance.drop(columns = ["SK_DPD", "AMT_DRAWINGS_ATM_CURRENT", "AMT_DRAWINGS_OTHER_CURRENT", "AMT_DRAWINGS_CURRENT", "AMT_DRAWINGS_POS_CURRENT", "CNT_DRAWINGS_ATM_CURRENT", "CNT_DRAWINGS_OTHER_CURRENT", "CNT_DRAWINGS_CURRENT", "CNT_DRAWINGS_POS_CURRENT", "AMT_RECEIVABLE_PRINCIPAL", "AMT_RECIVABLE", "AMT_PAYMENT_CURRENT", "MONTHS_BALANCE"])

contract_status_dict = {"Refused":1, "Demand":2, "Sent proposal":3, "Active": 4 , "Signed":5 ,"Completed": 6,"Approved":7}
credit_card_balance.NAME_CONTRACT_STATUS = credit_card_balance.NAME_CONTRACT_STATUS.map(contract_status_dict)

def compress_median(df_example, list_of_columns):
    for each_column in list_of_columns:
        df_example[each_column] = df_example.groupby(["SK_ID_PREV","SK_ID_CURR"])[each_column].transform(lambda x: x.median())
    return df_example.drop_duplicates()

credit_card_balance = compress_median(credit_card_balance, credit_card_balance.drop(columns = ["SK_ID_PREV","SK_ID_CURR"]).columns)
credit_card_balance.drop_duplicates(inplace=True)

## Previous Application

In [4]:
previous_application.drop(columns =["RATE_INTEREST_PRIMARY", "RATE_INTEREST_PRIVILEGED"], inplace=True)
previous_application.dropna(subset=["AMT_CREDIT", "PRODUCT_COMBINATION"], inplace=True)

previous_application.AMT_DOWN_PAYMENT = previous_application.AMT_DOWN_PAYMENT.interpolate(method='linear', x=previous_application.AMT_APPLICATION)
previous_application.RATE_DOWN_PAYMENT = previous_application.RATE_DOWN_PAYMENT.interpolate(method='linear', x=previous_application.AMT_DOWN_PAYMENT)
previous_application.CNT_PAYMENT = previous_application.CNT_PAYMENT.interpolate(method='linear', x=previous_application.AMT_APPLICATION)
previous_application.AMT_GOODS_PRICE  = previous_application.AMT_GOODS_PRICE.interpolate(method='linear', x=previous_application.AMT_APPLICATION)

previous_application.AMT_ANNUITY =  previous_application.groupby("NAME_CONTRACT_TYPE").AMT_ANNUITY.transform(lambda x: x.fillna(x.median()))
previous_application.NAME_TYPE_SUITE = previous_application.NAME_TYPE_SUITE.fillna("Unaccompanied")
previous_application.drop(columns = ["DAYS_FIRST_DRAWING", "DAYS_FIRST_DUE","DAYS_LAST_DUE_1ST_VERSION","DAYS_LAST_DUE","DAYS_TERMINATION","NFLAG_INSURED_ON_APPROVAL"], inplace=True)

weekday_encoding_map = {'TUESDAY':2, 'WEDNESDAY':3, 'MONDAY':1, 'FRIDAY':5, 'THURSDAY':4, 'SATURDAY':6, 'SUNDAY':7}
previous_application.WEEKDAY_APPR_PROCESS_START = previous_application.WEEKDAY_APPR_PROCESS_START.map(weekday_encoding_map)

yield_encoding_map = {'XNA':0, 'middle':3, 'high':4, 'low_normal':2, 'low_action':1}
previous_application.NAME_YIELD_GROUP = previous_application.NAME_YIELD_GROUP.map(yield_encoding_map)

def categorical_ordinal_encoding(df_example):
    df_example = df_example.copy()
    for each_column in df_example.select_dtypes(include="object").columns:
        map_dict = {}
        categories = list(df_example[each_column].value_counts(ascending=True).index)
        for each_category in categories:
            # for each_index in range(1, len(df_example[each_column].value_counts(ascending=True).index)):
            map_dict[each_category] = categories.index(each_category)
        df_example[each_column] = df_example[each_column].map(map_dict)
    return df_example

previous_application = categorical_ordinal_encoding(previous_application)

## FILL NA Simple

In [19]:
for each_column in application_test.select_dtypes(exclude="object").columns:
    application_test[each_column] = application_test[each_column].fillna(application_train[each_column].median())
    
for each_column in application_test.select_dtypes(include="object").columns:
    application_test[each_column] = application_test[each_column].fillna(application_train[each_column].mode()[0])

for each_column in application_train.select_dtypes(exclude="object").columns:
    application_train[each_column] = application_train[each_column].fillna(application_train[each_column].median())
    
for each_column in application_train.select_dtypes(include="object").columns:
    application_train[each_column] = application_train[each_column].fillna(application_train[each_column].mode()[0])

## Apply Same Encoding

In [22]:
import category_encoders as ce
one = ce.OneHotEncoder(use_cat_names=True)
combined_data = pd.concat([application_train.drop(columns ="TARGET"), application_test])
one.fit(combined_data)

OneHotEncoder(cols=['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',
                    'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE',
                    'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
                    'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
                    'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE',
                    'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE',
                    'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
              use_cat_names=True)

In [23]:
application_train_encoded = one.transform(application_train.drop(columns ="TARGET"))
application_train_encoded["TARGET"] = application_train.TARGET

In [24]:
application_test_encoded = one.transform(application_test)

## Merge Data

In [56]:
first_merge_df = installments_payments.merge(credit_card_balance, on=["SK_ID_PREV", "SK_ID_CURR"])
first_merge_df.shape

(72459, 16)

In [115]:
second_merge_df = first_merge_df.merge(previous_application, on=["SK_ID_PREV", "SK_ID_CURR"])
duplicated_SK_ID_CURRs = second_merge_df[second_merge_df.SK_ID_CURR.duplicated()].index
second_merge_df.drop(duplicated_SK_ID_CURRs, inplace=True)
second_merge_df.drop(columns="SK_ID_PREV", inplace=True)
second_merge_df.shape


(62076, 42)

In [122]:
train_merge = second_merge_df.merge(application_train_encoded, on="SK_ID_CURR")
train_merge.shape

(52962, 287)

In [121]:
test_merge = second_merge_df.merge(application_test_encoded, on="SK_ID_CURR", how="right")
# RIGHT MERGING APPLICATION TEST TO PRESERVE SK_ID_CURR
# HOWEVER, THIS RESULTS IN THOUSANDS OF NULLS (80% of Data, 39000 out of 48000)
# COMMENNTED OUT CODE FILLS IN NULLS, BUT NOT TREMENDOUSLY ACCURATE
# for each_column in test_merge.drop(columns = ["SK_ID_CURR","AMT_REQ_CREDIT_BUREAU_YEAR"]).columns:
#      test_merge[each_column] = test_merge.groupby("AMT_REQ_CREDIT_BUREAU_YEAR")[each_column].transform(lambda x: x.fillna(test_merge[each_column].mean()))
test_merge

,SK_ID_CURR,NUM_INSTALMENT_VERSION,DAYS_DIFFERENCE,AMT_DIFFERENCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_TOTAL_CURRENT,AMT_TOTAL_RECEIVABLE,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS_x,SK_DPD_DEF,AMT_DRAWINGS_TOTAL,CNT_DRAWINGS_TOTAL,COUNT_CREDIT_CARD,NAME_CONTRACT_TYPE,AMT_ANNUITY_x,AMT_APPLICATION,AMT_CREDIT_x,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE_x,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START_x,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS_y,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_F,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,FLAG_OWN_REALTY_N,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_y,AMT_ANNUITY_y,AMT_GOODS_PRICE_y,NAME_TYPE_SUITE_Unaccompanied,NAME_TYPE_SUITE_Family,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,NAME_TYPE_SUITE_Group of people,NAME_INCOME_TYPE_Working,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Maternity leave,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Academic degree,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Widow,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Unknown,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Co-op apartment,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_Waiters/barmen staff,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_HR staff,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,HOUR_APPR_PROCESS_START_y,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Trade: type 2

In [119]:
39630/48744 

0.8130231413096997

# Early Model

In [104]:
# SUCCESS WITH SIMPLE
X = application_train_encoded.drop(columns="TARGET")
y = application_train_encoded.TARGET

In [111]:
# MERGING MULTIPLE DATAFRAMES RESULTS IN NULLs FOR test_merge
X = train_merge.drop(columns="TARGET")
y = train_merge.TARGET

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

In [114]:
# LOGISTIC REGRESSION MODEL
lgr_model = LogisticRegression()
lgr_model.fit(X_train, y_train)
#lgr_model.predict_proba(X_test) # Predicted Probabilities

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [112]:
# XGB BOOST MODEL
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

# Early Entry

In [110]:
df_entry = pd.DataFrame([])
df_entry["SK_ID_CURR"] = application_test_encoded.SK_ID_CURR.astype("int")
df_entry["TARGET"] = np.round(xgb_model.predict_proba(application_test_encoded)[:,1],1)
df_entry
df_entry.to_csv("Test_Entry_XGB_7.csv", index =False)

# SCRAP

## TIP: More important to remove data than impute data

In [ ]:
# NOTE of Potentially Similar Columns

# Previous Application -- AMT_APPLICATION and AMT_CREDIT

# Credit Card Balance -- Several Options

# def application_encoding(df_example):
#     df_example = df_example.copy()
#     for each_column in df_example.select_dtypes(include="object").columns:
#         map_dict = {}
#         categories = list(df_example[each_column].value_counts(ascending=True).index)
#         for each_category in categories:
#             map_dict[each_category] = categories.index(each_category)
#         df_example[each_column] = df_example[each_column].map(map_dict)
        
#     return df_example

## Compress Columns: Engineering

DAYS_DIFFERENCE
* If entry payment has larger abs, then paid on time
* If negative, how many days late

AMT_DIFFERECE 
* If negative, then they did not fully pay the installment they were supposed to



NOTE: Opportuntiies to also merge POS_CASH_balance and Bureau 

In [ ]:
# bureau_df = bureau_balance.merge(bureau, on="SK_ID_BUREAU")
# bureau_df.groupby("SK_ID_BUREAU").STATUS.count().reset_index().sort_values(by="STATUS")
# MERGE BREAKS: third_merge_df.merge(bureau_df, on="SK_ID_CURR")

In [57]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     "n_estimators": [100, 200, 300],
#     "max_depth": [5, 10, 15],
#     "learning_rate": [0.01, 0.1, 0.2],
# }

# grid = GridSearchCV(xgb_model, param_grid, cv=3, verbose=1)
# grid.fit(X_train, y_train)
# grid.best_estimator_.predict(X_test)

ORDINAL ENCODE THE DATA IN PREVIOUS APPLICATION (Did it for Previous Application)

Simplify so dont need to one hot encode everything, control the amount of columns


For application_train 

Train encoder on training data

Then use encode for application_test

## IDEAS

Need to right merge with application test in order to preserve all SK_ID_CURR values

As it stands, test_merge has many nulls 

However, that leaves many NULLs where an SKID was not listed with previous information (Credit Card Balance does not include them all)

Tried to fill empty nulls with the mean for that category after the fact 

Aim to find a way to preserve information




More in depth way to handle the NULLs in General

Future: interpolate based on highest correlated column



Remove closely correlated variables

Add information from bureau and POS CASH 





# ACHEIVED TARGET

I achieved the best Score by not even including the other columns

Simple encoding of the application_train and application_test with an XG Boost Model

Tried to string together many other features, but ended up with many Nulls that didnt line up with the SK_ID_CURR of the application_test. THis made me impute many of the values, based on unspecific groupings